# Strategy Testing and Optimization

This notebook demonstrates how to test and optimize trading strategies using the BTB framework.

In [ ]:
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

# Add project root to path for imports
sys.path.append(os.path.abspath(".."))

from btb.backtest.engine import Backtester
from btb.backtest.metrics import calculate_metrics
from btb.backtest.monte_carlo import MonteCarloSimulation
from btb.backtest.walk_forward import WalkForwardAnalysis
from btb.data.loader import DataLoader
from btb.strategies.factory import create_strategy
from btb.utils.config import load_config


## 1. Load Configuration

In [ ]:
# Load backtest configuration
config = load_config("../config/backtest_config.yaml")
print(
    f"Backtest configuration loaded for {config['backtest']['symbols'][0]} with {config['backtest']['timeframes'][0]} timeframe"
)

## 2. Load Historical Data

In [ ]:
# Initialize data loader
data_config = {"use_dummy": True}
data_loader = DataLoader(data_config)

# Load historical market data
start_date = config["backtest"]["start_date"]
end_date = config["backtest"]["end_date"]
symbols = config["backtest"]["symbols"]
timeframes = config["backtest"]["timeframes"]
data = data_loader.load_data(symbols=symbols, timeframes=timeframes, start_date=start_date, end_date=end_date)

# Get the primary symbol and timeframe
symbol = symbols[0]
timeframe = timeframes[0]
df = data[f"{symbol}_{timeframe}"]

# Display data summary
print(f"Loaded {len(df)} data points from {df.index.min()} to {df.index.max()}")
df.head()

## 3. Initialize Strategy

In [ ]:
# Load models
models_dir = "../models/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize strategy from factory
strategy_type = config["backtest"]["strategy"]
strategy_params = config.get("strategy_params", {})

# Initialize strategy
strategy = create_strategy(strategy_type, strategy_params)

print(f"Initialized {strategy_type} strategy")

## 4. Run Basic Backtest

In [ ]:
# Create a full config for the backtester
backtester_config = {
    "backtest": {
        "initial_capital": config["backtest"]["initial_capital"],
        "commission": config["backtest"].get("commission", 0.0007),
        "slippage": config["backtest"].get("slippage", 0.0001),
        "strategy": strategy_type,
        "symbols": symbols,
        "timeframes": timeframes,
        "start_date": start_date,
        "end_date": end_date,
    },
    "strategy_params": strategy_params,
}

# Initialize backtester
backtester = Backtester(backtester_config)

# Run backtest
results = backtester.run()

# Convert results to DataFrame for easier analysis
equity_curve = results["equity_curve"]
trades = results["trades"]

# Create a DataFrame with the equity curve
results_df = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq="D"))
results_df["equity"] = pd.Series(equity_curve)
results_df = results_df.dropna()

# Calculate drawdown
peak = results_df["equity"].cummax()
results_df["drawdown"] = (results_df["equity"] - peak) / peak

# Display first few rows of results
results_df.head()

## 5. Analyze Backtest Performance

In [ ]:
# Calculate performance metrics
metrics = calculate_metrics(results)

# Display metrics
print("Performance Metrics:")
print(f"Total Return: {metrics['total_return']:.2f}%")
print(f"Annualized Return: {metrics['annualized_return']:.2f}%")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Sortino Ratio: {metrics['sortino_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2f}%")
print(f"Calmar Ratio: {metrics['calmar_ratio']:.2f}")
print(f"Win Rate: {metrics['win_rate']:.2f}%")
print(f"Profit Factor: {metrics['profit_factor']:.2f}")
print(f"Recovery Factor: {metrics['recovery_factor']:.2f}")
print(f"Risk of Ruin: {metrics['risk_of_ruin']:.4f}")

In [ ]:
# Plot equity curve
plt.figure(figsize=(14, 7))
plt.plot(results.index, results["equity"])
plt.title(f"{strategy_type} Strategy - Equity Curve")
plt.xlabel("Date")
plt.ylabel("Equity ($)")
plt.grid(True)
plt.show()

In [ ]:
# Plot drawdowns
plt.figure(figsize=(14, 7))
plt.plot(results.index, results["drawdown"] * 100)
plt.fill_between(results.index, results["drawdown"] * 100, alpha=0.3)
plt.title("Drawdown Percentage")
plt.xlabel("Date")
plt.ylabel("Drawdown (%)")
plt.grid(True)
plt.show()

In [ ]:
# Plot monthly returns
if "monthly_return" in results.columns:
    monthly_returns = results["monthly_return"].dropna()
else:
    # Calculate monthly returns if not provided in results
    monthly_returns = results["equity"].resample("M").last().pct_change().dropna()

plt.figure(figsize=(14, 7))
monthly_returns.plot(kind="bar")
plt.title("Monthly Returns")
plt.xlabel("Month")
plt.ylabel("Return (%)")
plt.grid(True, axis="y")
plt.show()

## 6. Analyze Trade Statistics

In [ ]:
# Get trade list
trades = backtester.get_trades()

if len(trades) > 0:
    # Display trade summary
    print(f"Total Trades: {len(trades)}")
    print(f"Winning Trades: {sum(1 for t in trades if t['pnl'] > 0)}")
    print(f"Losing Trades: {sum(1 for t in trades if t['pnl'] <= 0)}")

    # Convert trades to DataFrame for analysis
    trades_df = pd.DataFrame(trades)

    # Display first few trades
    trades_df.head()
else:
    print("No trades were executed during the backtest.")

In [ ]:
if len(trades) > 0:
    # Plot PnL distribution
    plt.figure(figsize=(12, 6))
    plt.hist(trades_df["pnl"], bins=30, alpha=0.7)
    plt.axvline(x=0, color="r", linestyle="--")
    plt.title("PnL Distribution")
    plt.xlabel("Profit/Loss")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

    # Plot trade holding periods
    plt.figure(figsize=(12, 6))
    trades_df["holding_period"] = (
        pd.to_datetime(trades_df["exit_time"]) - pd.to_datetime(trades_df["entry_time"])
    ).dt.total_seconds() / 3600  # in hours
    plt.hist(trades_df["holding_period"], bins=30, alpha=0.7)
    plt.title("Trade Holding Periods")
    plt.xlabel("Holding Period (hours)")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

## 7. Walk-Forward Analysis

In [ ]:
# Initialize walk-forward analysis
wfa = WalkForwardAnalysis(
    strategy=strategy,
    data=df,
    initial_capital=config["initial_capital"],
    train_size=int(config["walk_forward"]["train_size"]),  # days/candles in training window
    test_size=int(config["walk_forward"]["test_size"]),  # days/candles in testing window
    step_size=int(config["walk_forward"]["step_size"]),  # days/candles to step forward
)

# Run walk-forward analysis
wfa_results = wfa.run()

# Display walk-forward windows
print(f"Completed {len(wfa_results['windows'])} walk-forward windows")
for i, window in enumerate(wfa_results["windows"]):
    print(f"Window {i + 1}: {window['train_start']} to {window['test_end']} - Return: {window['return']:.2f}%")

In [ ]:
# Plot walk-forward equity curves
plt.figure(figsize=(14, 10))

# Plot combined equity curve
plt.subplot(2, 1, 1)
plt.plot(wfa_results["combined_equity"].index, wfa_results["combined_equity"])
plt.title("Walk-Forward Analysis - Combined Equity Curve")
plt.xlabel("Date")
plt.ylabel("Equity ($)")
plt.grid(True)

# Plot individual window returns
plt.subplot(2, 1, 2)
window_returns = [window["return"] for window in wfa_results["windows"]]
window_labels = [f"W{i + 1}" for i in range(len(window_returns))]
plt.bar(window_labels, window_returns)
plt.axhline(y=0, color="r", linestyle="--")
plt.title("Returns by Walk-Forward Window")
plt.xlabel("Window")
plt.ylabel("Return (%)")
plt.grid(True, axis="y")

plt.tight_layout()
plt.show()

## 8. Monte Carlo Simulation

In [ ]:
# Initialize Monte Carlo simulation
if len(trades) > 0:
    mc = MonteCarloSimulation(
        trades=trades,
        initial_capital=config["initial_capital"],
        num_simulations=config["monte_carlo"]["num_simulations"],
        confidence_level=config["monte_carlo"]["confidence_level"],
    )

    # Run simulation
    mc_results = mc.run()

    # Display Monte Carlo simulation results
    print("Monte Carlo Simulation Results:")
    print(f"Median Terminal Equity: ${mc_results['median_terminal_equity']:.2f}")
    print(
        f"Confidence Interval: ${mc_results['confidence_interval'][0]:.2f} to ${mc_results['confidence_interval'][1]:.2f}"
    )
    print(f"Expected Drawdown: {mc_results['expected_drawdown']:.2f}%")
    print(f"Risk of Ruin: {mc_results['risk_of_ruin']:.4f}")
else:
    print("Cannot run Monte Carlo simulation - no trades available.")

In [ ]:
if len(trades) > 0:
    # Plot Monte Carlo equity curves
    plt.figure(figsize=(14, 7))

    # Plot all simulations
    for sim in mc_results["equity_curves"][:100]:  # Plot first 100 simulations for clarity
        plt.plot(sim, alpha=0.1, color="blue")

    # Plot median curve
    plt.plot(mc_results["median_curve"], linewidth=2, color="black", label="Median")

    # Plot confidence interval
    plt.plot(
        mc_results["lower_bound"],
        linewidth=2,
        color="red",
        label=f"{config['monte_carlo']['confidence_level'] * 100}% Lower Bound",
    )
    plt.plot(
        mc_results["upper_bound"],
        linewidth=2,
        color="green",
        label=f"{config['monte_carlo']['confidence_level'] * 100}% Upper Bound",
    )

    plt.title("Monte Carlo Simulation - Equity Curves")
    plt.xlabel("Trade Number")
    plt.ylabel("Equity ($)")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot terminal equity distribution
    plt.figure(figsize=(14, 7))
    plt.hist(mc_results["terminal_equities"], bins=50, alpha=0.7)
    plt.axvline(x=mc_results["median_terminal_equity"], color="black", linestyle="-", linewidth=2, label="Median")
    plt.axvline(
        x=mc_results["confidence_interval"][0],
        color="red",
        linestyle="--",
        linewidth=2,
        label=f"{config['monte_carlo']['confidence_level'] * 100}% Lower Bound",
    )
    plt.axvline(
        x=mc_results["confidence_interval"][1],
        color="green",
        linestyle="--",
        linewidth=2,
        label=f"{config['monte_carlo']['confidence_level'] * 100}% Upper Bound",
    )
    plt.title("Terminal Equity Distribution")
    plt.xlabel("Terminal Equity ($)")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)
    plt.show()

## 9. Strategy Optimization

In [ ]:
def optimize_strategy_parameter(param_name, param_values, config):
    """Optimize a single strategy parameter"""
    results = []

    for value in param_values:
        # Create a copy of the config with modified parameter
        test_config = config.copy()

        # Update the parameter in the config
        if "." in param_name:  # Handle nested parameters
            parts = param_name.split(".")
            if len(parts) == 2:  # Only one level of nesting
                test_config[parts[0]][parts[1]] = value
        else:
            test_config[param_name] = value

        # Initialize strategy with new config
        test_strategy = create_strategy(
            strategy_type=strategy_type,
            config=test_config,
            model_path=f"{models_dir}{strategy_type}_model.pth",
            device=device,
        )

        # Initialize backtester
        test_backtester = Backtester(
            strategy=test_strategy,
            data=df,
            initial_capital=test_config["initial_capital"],
            position_size=test_config["position_size"],
            commission=test_config["commission"],
            slippage=test_config["slippage"],
        )

        # Run backtest
        test_results = test_backtester.run()

        # Calculate metrics
        test_metrics = calculate_metrics(test_results)

        # Record results
        results.append(
            {
                "parameter": param_name,
                "value": value,
                "total_return": test_metrics["total_return"],
                "sharpe_ratio": test_metrics["sharpe_ratio"],
                "max_drawdown": test_metrics["max_drawdown"],
                "win_rate": test_metrics["win_rate"],
                "profit_factor": test_metrics["profit_factor"],
            }
        )

    return pd.DataFrame(results)

In [ ]:
# Example parameter to optimize
param_name = "signal_threshold"  # This would be a parameter in your strategy
param_values = [0.0001, 0.0005, 0.001, 0.002, 0.005]

# Run optimization
# optimization_results = optimize_strategy_parameter(param_name, param_values, config)

# Display optimization results
# print("Optimization Results:")
# print(optimization_results)

# Plot optimization results
# plt.figure(figsize=(14, 7))
# plt.plot(optimization_results['value'], optimization_results['total_return'], marker='o')
# plt.title(f'Parameter Optimization - {param_name}')
# plt.xlabel(f'{param_name} Value')
# plt.ylabel('Total Return (%)')
# plt.grid(True)
# plt.show()

## 10. Position Sizing Optimization

In [ ]:
# Optimize position size
position_sizes = [0.05, 0.1, 0.2, 0.3, 0.5, 0.75, 1.0]  # Percentage of capital

# Run optimization
# pos_size_results = optimize_strategy_parameter('position_size', position_sizes, config)

# Plot position sizing results
# plt.figure(figsize=(14, 10))

# Plot return vs position size
# plt.subplot(2, 1, 1)
# plt.plot(pos_size_results['value'], pos_size_results['total_return'], marker='o')
# plt.title('Total Return vs Position Size')
# plt.xlabel('Position Size (% of Capital)')
# plt.ylabel('Total Return (%)')
# plt.grid(True)

# Plot drawdown vs position size
# plt.subplot(2, 1, 2)
# plt.plot(pos_size_results['value'], pos_size_results['max_drawdown'], marker='o')
# plt.title('Max Drawdown vs Position Size')
# plt.xlabel('Position Size (% of Capital)')
# plt.ylabel('Max Drawdown (%)')
# plt.grid(True)

# plt.tight_layout()
# plt.show()

## 11. Save Optimized Configuration

In [ ]:
# Save optimized configuration to file
# from btb.utils.config import save_config

# Uncomment to save the optimized configuration
# optimized_config = config.copy()
# optimized_config['position_size'] = 0.2  # Example optimized value
# optimized_config['signal_threshold'] = 0.001  # Example optimized value

# save_config(optimized_config, '../config/optimized_backtest_config.yaml')
# print("Optimized configuration saved to '../config/optimized_backtest_config.yaml'")

## 12. Summary and Next Steps

In this notebook, we:

1. Loaded historical market data and trading configuration
2. Initialized and backtested a trading strategy
3. Analyzed backtest performance using various metrics and visualizations
4. Examined individual trade statistics
5. Performed walk-forward analysis to test strategy robustness
6. Used Monte Carlo simulation to assess risk and expected outcomes
7. Demonstrated how to optimize strategy parameters and position sizing

Next steps:
- Apply the optimized strategy in a live trading environment
- Continuously monitor performance and compare to backtest results
- Refine risk management and position sizing based on live performance
- Explore additional markets and timeframes for the strategy